# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [898]:
%load_ext sql
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)

# pd.read_sql用のコネクタ
conn = psycopg2.connect(**pgconfig)
# pd.to_sql用のcreate engine
engine = create_engine(dsl)
# MagicコマンドでSQLを書くための設定
%sql $dsl

df_customer = pd.read_sql(sql='select * from customer', con=conn)
df_category = pd.read_sql(sql='select * from category', con=conn)
df_product = pd.read_sql(sql='select * from product', con=conn)
df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
df_store = pd.read_sql(sql='select * from store', con=conn)
df_geocode = pd.read_sql(sql='select * from geocode', con=conn)
print("done")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
done


# 演習問題

---
> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [12]:
df_receipt.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
6,20181205,1259971200,S14024,1102,2,CS024514000042,P080101005,1,30
7,20190922,1285113600,S14040,1102,1,CS040415000178,P070501004,1,128
8,20170504,1209859200,S13020,1112,2,ZZ000000000000,P071302010,1,770
9,20191010,1286668800,S14027,1102,1,CS027514000015,P071101003,1,680


---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [16]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

,sales_ymd,customer_id,product_cd,amount
0,20181103,CS006214000001,P070305012,158
1,20181118,CS008415000097,P070701017,81
2,20170712,CS028414000014,P060101005,170
3,20190205,ZZ000000000000,P050301001,25
4,20180821,CS025415000050,P060102007,90
5,20190605,CS003515000195,P050102002,138
6,20181205,CS024514000042,P080101005,30
7,20190922,CS040415000178,P070501004,128
8,20170504,ZZ000000000000,P071302010,770
9,20191010,CS027514000015,P071101003,680


---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [29]:
df_004 = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
df_004[df_004['customer_id'] == 'CS018205000001']

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [45]:
df_005 = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
df_005[(df_005['customer_id'] == 'CS018205000001') & (df_005['amount'] >= 1000)]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [46]:
df_006 = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']]
df_006[(df_006['customer_id'] == 'CS018205000001') & ((df_006['amount'] >= 1000)| (df_006['quantity'] >= 5))]

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [47]:
df_007 = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
df_007[(df_007['customer_id'] == 'CS018205000001') & ((df_007['amount'] >= 1000) & (df_007['amount'] <= 2000))]

,sales_ymd,customer_id,product_cd,amount
72254,20180911,CS018205000001,P071401005,1100


---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [54]:
df_008 = df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']]
df_008[(df_008['customer_id'] == 'CS018205000001') & (df_008['product_cd'] != "P071401019")]

,sales_ymd,customer_id,product_cd,amount
36,20180911,CS018205000001,P071401012,2200
9843,20180414,CS018205000001,P060104007,600
21110,20170614,CS018205000001,P050206001,990
27673,20170614,CS018205000001,P060702015,108
27840,20190216,CS018205000001,P071005024,102
28757,20180414,CS018205000001,P071101002,278
39256,20190226,CS018205000001,P070902035,168
58121,20190924,CS018205000001,P060805001,495
68117,20190226,CS018205000001,P071401020,2200
72254,20180911,CS018205000001,P071401005,1100


---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [85]:
df_store[((df_store['prefecture_cd'] != '13') & (df_store['floor_area'] < 900))]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
38,S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.0220,35.66122,808.0


---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [93]:
df_store[df_store['store_cd'].str.contains('S14')].head(10)

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
19,S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [96]:
df_customer[df_customer['customer_id'].str.endswith('1')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
31,CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
56,CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
59,CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
63,CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
74,CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
85,CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
102,CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [98]:
df_store[df_store['address'].str.contains('横浜市')]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
18,S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
20,S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
26,S14048,中川中央店,14,神奈川県,神奈川県横浜市都筑区中川中央二丁目,カナガワケンヨコハマシツヅキクナカガワチュウオウニチョウメ,045-123-4051,139.5758,35.54912,1657.0
40,S14042,新山下店,14,神奈川県,神奈川県横浜市中区新山下二丁目,カナガワケンヨコハマシナカクシンヤマシタニチョウメ,045-123-4047,139.6593,35.43894,1044.0


---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [135]:
df_customer[df_customer['status_cd'].str.contains('[A-F]...........')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
6,CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
15,CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
21,CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
32,CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
33,CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
40,CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
44,CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
53,CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [133]:
df_customer[df_customer['status_cd'].str.contains('...........[1-9]')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
9,CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
14,CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
16,CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
22,CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
23,CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
24,CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
27,CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
28,CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [136]:
df_customer[df_customer['status_cd'].str.contains('[A-F]..........[1-9]')].head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
12,CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
68,CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
71,CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
122,CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
144,CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
178,CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
252,CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
259,CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
293,CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
297,CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [138]:
df_store[df_store['tel_no'].str.contains('...-...-....')]

,store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
0,S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698.0
1,S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
2,S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
3,S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
4,S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
7,S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
9,S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
11,S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087.0
12,S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
16,S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0


---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [141]:
df_customer.sort_values('birth_day', ascending=True).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
18817,CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
12328,CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
15682,CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
15302,CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
1681,CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
7511,CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
2378,CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
4680,CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
16070,CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0
6305,CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0


---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [142]:
df_customer.sort_values('birth_day', ascending=False).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
3670,CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
12493,CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
15977,CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
5716,CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
15097,CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [251]:
df_019_0 = df_receipt[['customer_id', 'amount']].sort_values('amount', ascending=False).head(10)
df_019_1 = np.array(df_019_0.rank(method='min', ascending=False)['amount']).astype('int64')
df_019_0['rank'] = df_019_1
df_019_0

,customer_id,amount,rank
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
72747,ZZ000000000000,5480,4
4632,CS015515000034,5480,4
10320,ZZ000000000000,5480,4
28304,ZZ000000000000,5440,7
97294,CS021515000089,5440,7
77874,CS008513000063,5280,9
55795,CS032414000072,5280,9


---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [254]:
df_020_0 = df_receipt[['customer_id', 'amount']].sort_values('amount', ascending=False).head(10)
df_020_1 = np.array(df_019_0.rank(method='first', ascending=False, numeric_only=True)['amount']).astype('int64')
df_020_0['rank'] = df_020_1
df_020_0

,customer_id,amount,rank
1202,CS011415000006,10925,1
62317,ZZ000000000000,6800,2
54095,CS028605000002,5780,3
72747,ZZ000000000000,5480,4
4632,CS015515000034,5480,5
10320,ZZ000000000000,5480,6
28304,ZZ000000000000,5440,7
97294,CS021515000089,5440,8
77874,CS008513000063,5280,9
55795,CS032414000072,5280,10


---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [257]:
df_receipt.shape[0]

104681

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [260]:
len(set(df_receipt['customer_id']))

8307

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [332]:
store_cd_023 = set(df_receipt['store_cd'])

result_023 = []
for st_cd in store_cd_023:
    st_cd_filter = df_receipt[df_receipt['store_cd'] == st_cd]
    
    amount_sum = np.sum(st_cd_filter['amount'])
    quantity_sum = np.sum(st_cd_filter['quantity'])
    
    result_023.append([st_cd, amount_sum, quantity_sum])
    
result_023

[['S14024', 736323, 2417],
 ['S14033', 725318, 2282],
 ['S13016', 793773, 2432],
 ['S13037', 693087, 2344],
 ['S13044', 520764, 1729],
 ['S13051', 107452, 354],
 ['S14040', 701858, 2233],
 ['S13015', 780873, 2248],
 ['S13017', 748221, 2376],
 ['S13004', 779373, 2390],
 ['S14011', 805724, 2434],
 ['S13009', 808870, 2486],
 ['S12030', 684402, 2403],
 ['S13039', 611888, 1981],
 ['S12013', 787513, 2425],
 ['S13043', 587895, 1881],
 ['S13035', 715869, 2219],
 ['S13031', 705968, 2336],
 ['S14026', 824537, 2503],
 ['S12007', 638761, 2099],
 ['S14025', 755581, 2394],
 ['S14027', 714550, 2303],
 ['S14006', 712839, 2284],
 ['S13003', 764294, 2197],
 ['S14023', 727630, 2258],
 ['S14036', 203694, 635],
 ['S13018', 790535, 2562],
 ['S13038', 708884, 2337],
 ['S14047', 338329, 1041],
 ['S14048', 234276, 769],
 ['S13032', 790501, 2491],
 ['S12014', 725167, 2358],
 ['S14046', 412646, 1354],
 ['S13041', 728266, 2233],
 ['S14010', 790361, 2290],
 ['S13008', 809288, 2491],
 ['S14028', 786145, 2458],
 ['S

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [334]:
store_cd_024 = list(set(df_receipt['customer_id']))[:10]

result_024 = []
for st_cd in store_cd_024:
    latest_date = int(df_receipt[df_receipt['customer_id'] == st_cd]['sales_ymd'].sort_values(ascending=False).head(1))
    result_024.append([st_cd, latest_date])

result_024

[['CS023214000005', 20190928],
 ['CS013512000023', 20180713],
 ['CS001515000060', 20180702],
 ['CS020414000113', 20190727],
 ['CS040215000052', 20180520],
 ['CS019515000174', 20190728],
 ['CS012615000131', 20190801],
 ['CS018511000029', 20181227],
 ['CS001814000018', 20171023],
 ['CS037514000078', 20190315]]

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [335]:
store_cd_025 = list(set(df_receipt['customer_id']))[:10]

result_025 = []
for st_cd in store_cd_025:
    oldest_date = int(df_receipt[df_receipt['customer_id'] == st_cd]['sales_ymd'].sort_values(ascending=True).head(1))
    result_025.append([st_cd, oldest_date])

result_025

[['CS023214000005', 20180103],
 ['CS013512000023', 20171103],
 ['CS001515000060', 20180702],
 ['CS020414000113', 20180509],
 ['CS040215000052', 20180418],
 ['CS019515000174', 20170819],
 ['CS012615000131', 20190801],
 ['CS018511000029', 20181227],
 ['CS001814000018', 20171023],
 ['CS037514000078', 20170819]]

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [336]:
store_cd_026 = list(set(df_receipt['customer_id']))

result_026 = []
for st_cd in store_cd_026:
    df_026 = df_receipt[df_receipt['customer_id'] == st_cd]['sales_ymd'].sort_values(ascending=False)
    
    latest_date = int(df_026.head(1))
    oldest_date = int(df_026.tail(1))
    result_026.append([st_cd, latest_date, oldest_date])
    
    if len(result_026) == 10:
        break
    else:
        pass

result_026

[['CS023214000005', 20190928, 20180103],
 ['CS013512000023', 20180713, 20171103],
 ['CS001515000060', 20180702, 20180702],
 ['CS020414000113', 20190727, 20180509],
 ['CS040215000052', 20180520, 20180418],
 ['CS019515000174', 20190728, 20170819],
 ['CS012615000131', 20190801, 20190801],
 ['CS018511000029', 20181227, 20181227],
 ['CS001814000018', 20171023, 20171023],
 ['CS037514000078', 20190315, 20170819]]

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [370]:
store_cd_027 = set(df_receipt["store_cd"])

result_027 = []
for st_cd in store_cd_027:
    result_027.append([st_cd, np.mean(df_receipt[df_receipt['store_cd'] == st_cd]['amount'])])

result_027 = pd.DataFrame(result_027).sort_values(by=1, ascending=True)
result_027.head(5)

,0,1
45,S14042,278.338886
12,S12030,288.533727
41,S14050,290.591304
50,S14049,297.816774
3,S13037,299.907832


---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [379]:
store_cd_028 = set(df_receipt["store_cd"])

result_028 = []
for st_cd in store_cd_028:
    result_028.append([st_cd, np.median(df_receipt[df_receipt['store_cd'] == st_cd]['amount'])])

result_028 = pd.DataFrame(result_028).sort_values(by=1, ascending=True)
result_028.head(5)

,0,1
50,S14049,148.0
48,S14034,158.0
32,S14046,158.0
15,S13043,158.0
37,S13002,158.0


---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [384]:
from scipy.stats import mode

store_cd_029 = set(df_receipt["store_cd"])

result_029 = []
for st_cd in store_cd_029:
    result_029.append([st_cd, mode(df_receipt[df_receipt['store_cd'] == st_cd]['amount'])[0][0]])

result_029 = pd.DataFrame(result_029).sort_values(by=1, ascending=False)
result_029

,0,1
39,S13020,1100
33,S13041,1100
23,S13003,1100
6,S14040,298
0,S14024,148
14,S12013,148
50,S14049,148
36,S14028,148
32,S14046,148
30,S13032,148


---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [389]:
store_cd_030 = set(df_receipt["store_cd"])

result_030 = []
for st_cd in store_cd_030:
    result_030.append([st_cd, np.var(df_receipt[df_receipt['store_cd'] == st_cd]['amount'], axis=0)])

result_030 = pd.DataFrame(result_030).sort_values(by=1, ascending=True)
result_030.head(5)

,0,1
41,S14050,157698.130359
45,S14042,172061.187082
50,S14049,180361.837396
12,S12030,185542.898104
4,S13044,189597.952968


---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [390]:
store_cd_031 = set(df_receipt["store_cd"])

result_031 = []
for st_cd in store_cd_031:
    result_031.append([st_cd, np.std(df_receipt[df_receipt['store_cd'] == st_cd]['amount'], axis=0)])

result_031 = pd.DataFrame(result_031).sort_values(by=1, ascending=True)
result_031.head(5)

,0,1
41,S14050,397.112239
45,S14042,414.802588
50,S14049,424.690284
12,S12030,430.746907
4,S13044,435.428471


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [402]:
a_032 = np.array(df_receipt['amount'])
print({'0%':np.percentile(a_032,0), '25%':np.percentile(a_032,25), '50%':np.percentile(a_032,50), \
        '75%':np.percentile(a_032,75), '100%':np.percentile(a_032,100)})

{'0%': 10.0, '25%': 102.0, '50%': 170.0, '75%': 288.0, '100%': 10925.0}


---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [407]:
store_cd_033 = set(df_receipt["store_cd"])

result_033 = []
for st_cd in store_cd_033:
    result_033.append([st_cd, np.mean(df_receipt[df_receipt['store_cd'] == st_cd]['amount'])])

result_033 = pd.DataFrame(result_033).sort_values(by=1, ascending=True)
result_033[result_033[1] >= 330]

,0,1
28,S14047,330.077073
40,S14045,330.082073
14,S12013,330.194130
38,S13019,330.208616
9,S13004,330.943949
18,S14026,332.340588
10,S14011,335.718333
39,S13020,337.879932
42,S13001,348.470386
34,S14010,348.791262


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [445]:
df_receipt_034 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_034 = set(df_receipt_034['customer_id'])

result_034 = []
for c_id in customer_set_034:
    c_am = df_receipt_034[df_receipt_034['customer_id'] == c_id]['amount']
    result_034.append([c_id, np.sum(c_am), np.mean(c_am)])

c_sum = np.array(result_034)[:,1].astype('int64')
c_mean = np.mean(c_sum)
c_mean


2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [447]:
df_receipt_035 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_035 = set(df_receipt_034['customer_id'])

result_035 = []
for c_id in customer_set_035:
    c_am = df_receipt_035[df_receipt_035['customer_id'] == c_id]['amount']
    result_035.append([c_id, np.sum(c_am), np.mean(c_am)])

result_035 = np.array(result_035)
c_sum = result_035[:,1].astype('int64')
c_mean = np.mean(c_sum)

result_035[c_sum >= c_mean][:10,:2]

array([['CS023214000005', '4969'],
       ['CS020414000113', '5047'],
       ['CS019515000174', '4179'],
       ['CS021515000211', '10148'],
       ['CS027414000023', '4098'],
       ['CS015415000068', '12315'],
       ['CS031415000086', '4535'],
       ['CS017412000025', '3607'],
       ['CS022513000145', '4680'],
       ['CS026415000080', '3419']], dtype='<U18')

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [491]:
df_036 = pd.merge(df_receipt, df_store, how='inner')
df_036[['amount', 
        'customer_id', 
        'product_cd', 
        'quantity', 
        'receipt_no', 
        'receipt_sub_no', 
        'sales_epoch', 
        'sales_ymd', 
        'store_cd', 
        'store_name']].head(10)

,amount,customer_id,product_cd,quantity,receipt_no,receipt_sub_no,sales_epoch,sales_ymd,store_cd,store_name
0,158,CS006214000001,P070305012,1,112,1,1257206400,20181103,S14006,葛が谷店
1,48,ZZ000000000000,P080401001,1,112,2,1258329600,20181116,S14006,葛が谷店
2,220,CS006815000006,P050406035,1,1162,1,1200614400,20170118,S14006,葛が谷店
3,80,CS006514000034,P060104003,1,1192,1,1274659200,20190524,S14006,葛が谷店
4,148,ZZ000000000000,P060501002,1,112,2,1271635200,20190419,S14006,葛が谷店
5,88,ZZ000000000000,P050701001,1,1152,2,1258588800,20181119,S14006,葛が谷店
6,80,CS006515000175,P090903001,1,1132,2,1228953600,20171211,S14006,葛が谷店
7,405,CS006415000221,P040602001,1,1112,2,1287619200,20191021,S14006,葛が谷店
8,330,CS006411000036,P090301051,1,1132,2,1215648000,20170710,S14006,葛が谷店
9,115,CS006211000012,P050104001,1,112,1,1280966400,20190805,S14006,葛が谷店


---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [492]:
df_037 = pd.merge(df_product, df_category, how='inner')
df_037[['category_major_cd',
        'category_medium_cd',
        'category_small_cd',
        'product_cd',
        'unit_cost',
        'unit_price',
        'category_small_name']].head(10)

,category_major_cd,category_medium_cd,category_small_cd,product_cd,unit_cost,unit_price,category_small_name
0,04,0401,040101,P040101001,149.0,198.0,弁当類
1,04,0401,040101,P040101002,164.0,218.0,弁当類
2,04,0401,040101,P040101003,173.0,230.0,弁当類
3,04,0401,040101,P040101004,186.0,248.0,弁当類
4,04,0401,040101,P040101005,201.0,268.0,弁当類
5,04,0401,040101,P040101006,224.0,298.0,弁当類
6,04,0401,040101,P040101007,254.0,338.0,弁当類
7,04,0401,040101,P040101008,315.0,420.0,弁当類
8,04,0401,040101,P040101009,374.0,498.0,弁当類
9,04,0401,040101,P040101010,435.0,580.0,弁当類


---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [489]:
df_038 = pd.merge(df_customer, df_receipt)
df_038 = df_038[df_038['gender_cd'] == '1']
df_038 = df_038[df_038['customer_id'].str.contains('[A-Y]...........')]

customer_038 = set(df_038['customer_id'])

result_038 = []
for c_id in customer_038:
    c_am = df_038[df_038['customer_id'] == c_id]['amount']
    result_038.append([c_id, np.sum(c_am)])
    
result_038 = np.array(result_038)
result_038[:10]

array([['CS023214000005', '4969'],
       ['CS001515000060', '168'],
       ['CS013512000023', '623'],
       ['CS020414000113', '5047'],
       ['CS040215000052', '461'],
       ['CS019515000174', '4179'],
       ['CS012615000131', '231'],
       ['CS018511000029', '2320'],
       ['CS037514000078', '824'],
       ['CS001814000018', '208']], dtype='<U14')

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [497]:
df_039 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_039 = set(df_039['customer_id'])

result = 
for c_id in customer_set_039:
    

SyntaxError: invalid syntax (<ipython-input-497-8766e05e32c9>, line 4)

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [511]:
#cheat
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()

df_store_tmp['key'] = 0
df_product_tmp['key'] = 0

len(pd.merge(df_store_tmp, df_product_tmp, on='key', how='outer'))

531590

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [542]:
date_list = list(set(df_receipt['sales_ymd']))
date_list.sort()

result_041 = []
for date in date_list:
    result_041.append([date, np.sum(df_receipt[df_receipt['sales_ymd'] == date]['amount'])])
    
result_041 = np.array(result_041)
res_diff = np.diff(result_041[:,1])
res_diff = np.insert(res_diff, 0, 0)
np.c_[result_041[:], res_diff][:10]

array([[20170101,    33723,        0],
       [20170102,    24165,    -9558],
       [20170103,    27503,     3338],
       [20170104,    36165,     8662],
       [20170105,    37830,     1665],
       [20170106,    32387,    -5443],
       [20170107,    23415,    -8972],
       [20170108,    24737,     1322],
       [20170109,    26718,     1981],
       [20170110,    20143,    -6575]])

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [583]:
date_diff = 3
date_list = list(set(df_receipt['sales_ymd']))
date_list.sort()

result_042 = []
for date in date_list:
    result_042.append([date, np.sum(df_receipt[df_receipt['sales_ymd'] == date]['amount'])])
    
result_042 = np.array(result_042)

base = np.zeros([1,4])
for num, con in enumerate([np.r_[np.array([[0,0] for x in range(i+1)]), result_042] for i in range(date_diff)]):
    base = np.r_[base, np.c_[result_042, con[:-(num+1)]]]
    
for x in range(base.shape[1]):
    base = base[base[:,x] != 0]
    
base = base[base[:,2].argsort(), :]
base = base[base[:,0].argsort(), :][:10]

base

array([[20170102.,    24165., 20170101.,    33723.],
       [20170103.,    27503., 20170101.,    33723.],
       [20170103.,    27503., 20170102.,    24165.],
       [20170104.,    36165., 20170101.,    33723.],
       [20170104.,    36165., 20170102.,    24165.],
       [20170104.,    36165., 20170103.,    27503.],
       [20170105.,    37830., 20170102.,    24165.],
       [20170105.,    37830., 20170103.,    27503.],
       [20170105.,    37830., 20170104.,    36165.],
       [20170106.,    32387., 20170104.,    36165.]])

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [632]:
df_043 = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')

age_min = int(min(df_043['age'])/10)
age_max = int(max(df_043['age'])/10)
gender_list = list(set(df_043['gender_cd']))
gender_list.sort()
generation_list = [gen*10 for gen in range(age_min,age_max+1)]

result_043_0 = []
for gender in gender_list:
    result_043_1 = []
    for gen in generation_list:
        fil = df_043[((df_043['age'] < gen+10) & (df_043['age'] >= gen)&(df_043['gender_cd'] == gender))]
        result_043_1.append(np.sum(fil['amount']))
    
    result_043_0.append(result_043_1)
    
result_043_0 = np.array(result_043_0).T
result_043_0 = np.c_[np.array(generation_list), result_043_0]
result_043_0 = pd.DataFrame(result_043_0, columns=['era', 'male', 'female', 'unknown'])
df_sales_summary = result_043_0.replace(0, np.nan)
df_sales_summary


,era,male,female,unknown
0,10,1591.0,149836,4317.0
1,20,72940.0,1363724,44328.0
2,30,177322.0,693047,50441.0
3,40,19355.0,9320791,483512.0
4,50,54320.0,6685192,342923.0
5,60,272469.0,987741,71418.0
6,70,13435.0,29764,2427.0
7,80,46360.0,262923,5111.0
8,90,NaN,6260,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [633]:
#cheat
df_sales_summary = df_sales_summary.set_index('era'). \
        stack().reset_index().replace({'female':'01',
                                        'male':'00',
                                        'unknown':'99'}).rename(columns={'level_1':'gender_cd', 0: 'amount'})
df_sales_summary

,era,gender_cd,amount
0,10,00,1591.0
1,10,01,149836.0
2,10,99,4317.0
3,20,00,72940.0
4,20,01,1363724.0
5,20,99,44328.0
6,30,00,177322.0
7,30,01,693047.0
8,30,99,50441.0
9,40,00,19355.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [659]:
df_045 = df_customer[['birth_day', 'customer_id']]
df_045 = pd.DataFrame([pd.to_datetime(df_045['birth_day']).dt.strftime('%Y%m%d'), df_045['customer_id']])
df_045.T.head(10)

,birth_day,customer_id
0,19810429,CS021313000114
1,19520401,CS037613000071
2,19761004,CS031415000172
3,19330327,CS028811000001
4,19950329,CS001215000145
5,19740915,CS020401000016
6,19770809,CS015414000103
7,19730817,CS029403000008
8,19310502,CS015804000004
9,19620711,CS033513000180


---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [695]:
df_046 = df_customer[['application_date', 'customer_id']]
df_046 = df_046.assign(tra_application_date = df_046['application_date'].astype('datetime64'))
df_046.drop('application_date', axis=1).rename(columns={'tra_application_date':'application_date'}).head(10)

,customer_id,application_date
0,CS021313000114,2015-09-05
1,CS037613000071,2015-04-14
2,CS031415000172,2015-05-29
3,CS028811000001,2016-01-15
4,CS001215000145,2017-06-05
5,CS020401000016,2015-02-25
6,CS015414000103,2015-07-22
7,CS029403000008,2015-05-15
8,CS015804000004,2015-06-07
9,CS033513000180,2015-07-28


---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [703]:
df_047 = df_receipt[['sales_ymd', 'receipt_no', 'receipt_sub_no']]

df_047 = df_047.assign(tra_sales_ymd = df_047['sales_ymd'].astype('str').astype('datetime64'))
df_047.drop('sales_ymd', axis=1).rename(columns={'tra_sales_ymd':'sales_ymd'}).head(10)

,receipt_no,receipt_sub_no,sales_ymd
0,112,1,2018-11-03
1,1132,2,2018-11-18
2,1102,1,2017-07-12
3,1132,1,2019-02-05
4,1102,2,2018-08-21
5,1112,1,2019-06-05
6,1102,2,2018-12-05
7,1102,1,2019-09-22
8,1112,2,2017-05-04
9,1102,1,2019-10-10


---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [727]:
df_048 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']]
df_048 = df_048.assign(tra_sales_epoch = pd.to_datetime(df_048['sales_epoch'].astype(int), unit='s'))
df_048.drop('sales_epoch', axis=1).rename(columns={'tra_sales_epoch':'sales_epoch'}).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009-11-03
1,1132,2,2009-11-18
2,1102,1,2008-07-12
3,1132,1,2010-02-05
4,1102,2,2009-08-21
5,1112,1,2010-06-05
6,1102,2,2009-12-05
7,1102,1,2010-09-22
8,1112,2,2008-05-04
9,1102,1,2010-10-10


---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [712]:
df_049 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']]
df_049 = df_049.assign(tra_sales_epoch = pd.to_datetime(df_049['sales_epoch'].astype(int), unit='s').dt.year)
df_049.drop('sales_epoch', axis=1).rename(columns={'tra_sales_epoch':'sales_epoch'}).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,2009
1,1132,2,2009
2,1102,1,2008
3,1132,1,2010
4,1102,2,2009
5,1112,1,2010
6,1102,2,2009
7,1102,1,2010
8,1112,2,2008
9,1102,1,2010


---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [730]:
df_050 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']]
df_050 = df_050.assign(tra_sales_epoch = pd.to_datetime(df_050['sales_epoch'], unit='s').dt.strftime('%m'))
df_050.drop('sales_epoch', axis=1).rename(columns={'tra_sales_epoch':'sales_epoch'}).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,11
1,1132,2,11
2,1102,1,07
3,1132,1,02
4,1102,2,08
5,1112,1,06
6,1102,2,12
7,1102,1,09
8,1112,2,05
9,1102,1,10


---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [731]:
df_051 = df_receipt[['sales_epoch', 'receipt_no', 'receipt_sub_no']]
df_051 = df_051.assign(tra_sales_epoch = pd.to_datetime(df_051['sales_epoch'], unit='s').dt.strftime('%d'))
df_051.drop('sales_epoch', axis=1).rename(columns={'tra_sales_epoch':'sales_epoch'}).head(10)

,receipt_no,receipt_sub_no,sales_epoch
0,112,1,03
1,1132,2,18
2,1102,1,12
3,1132,1,05
4,1102,2,21
5,1112,1,05
6,1102,2,05
7,1102,1,22
8,1112,2,04
9,1102,1,10


---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [745]:

df_052 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_052 = set(df_052['customer_id'])

result_052= []
for c_id in customer_set_052:
    c_am = df_052[df_052['customer_id'] == c_id]['amount']
    result_052.append([c_id, np.sum(c_am)])
    
result_052 = np.array(result_052)

c_sum = result_052[:,1].astype('int64')
amount_binary = np.zeros_like(c_sum)
amount_binary[c_sum >= 2000] += 1
result_052 = np.c_[result_052, amount_binary]
result_052[:10]

array([['CS023214000005', '4969', '1'],
       ['CS013512000023', '623', '0'],
       ['CS001515000060', '168', '0'],
       ['CS020414000113', '5047', '1'],
       ['CS040215000052', '461', '0'],
       ['CS019515000174', '4179', '1'],
       ['CS012615000131', '231', '0'],
       ['CS018511000029', '2320', '1'],
       ['CS001814000018', '208', '0'],
       ['CS037514000078', '824', '0']], dtype='<U21')

---
> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [835]:
df_customer_tmp = df_customer.copy()
df_customer_tmp.loc[(df_customer_tmp['postal_cd'].str.contains('1[0-9][0-9]-....')) , 'postal_cd'] = '1'
df_customer_tmp.loc[(df_customer_tmp['postal_cd'].str.contains('20[0-9]-....')) , 'postal_cd'] = '1'
df_customer_tmp.loc[df_customer_tmp['postal_cd'] != '1' , 'postal_cd'] = '0'
df_053 = pd.merge(df_customer_tmp, df_receipt, on='customer_id', how='inner')

other = len(set(df_053[(df_053['postal_cd'] == '0')]['customer_id']))
tokyo = len(set(df_053[(df_053['postal_cd'] == '1')]['customer_id']))
{'0':other, '1':tokyo}

{'0': 3906, '1': 4400}

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [852]:
df_customer_tmp = df_customer.copy()[['customer_id', 'address']]
df_customer_tmp['address_1'] = df_customer_tmp['address']
for num, pref in enumerate(['埼玉県','千葉県','東京都','神奈川県']):
    df_customer_tmp.loc[(df_customer_tmp['address_1'].str.startswith(pref)) , 'address_1'] = str(10+num+1)
    
df_customer_tmp.head(10)

,customer_id,address,address_1
0,CS021313000114,神奈川県伊勢原市粟窪**********,14
1,CS037613000071,東京都江東区南砂**********,13
2,CS031415000172,東京都渋谷区代々木**********,13
3,CS028811000001,神奈川県横浜市泉区和泉町**********,14
4,CS001215000145,東京都大田区仲六郷**********,13
5,CS020401000016,東京都板橋区若木**********,13
6,CS015414000103,東京都江東区北砂**********,13
7,CS029403000008,千葉県浦安市海楽**********,12
8,CS015804000004,東京都江東区北砂**********,13
9,CS033513000180,神奈川県横浜市旭区善部町**********,14


---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [873]:
customer_set_055 = list(set(df_receipt['customer_id']))
customer_set_055.sort()

result_055 = []
for c_id in customer_set_055:
    c_am = df_receipt[df_receipt['customer_id'] == c_id]['amount']
    result_055.append([c_id, np.sum(c_am)])
    
result_055 = np.array(result_055)
result_055

c_sum = result_055[:,1].astype('int64')
c_per = np.percentile(c_sum, q=[25, 50, 75])

result_055 = np.c_[result_055, np.zeros_like(c_sum)]
result_055[:,2][c_sum < c_per[0]] = 1
result_055[:,2][np.logical_and(c_sum >= c_per[0], c_sum < c_per[1])] = 2
result_055[:,2][np.logical_and(c_sum >= c_per[1], c_sum < c_per[2])] = 3
result_055[:,2][c_sum >= c_per[2]] = 4
result_055[:10]

array([['CS001113000004', '1298', '2'],
       ['CS001114000005', '626', '2'],
       ['CS001115000010', '3044', '3'],
       ['CS001205000004', '1988', '3'],
       ['CS001205000006', '3337', '3'],
       ['CS001211000025', '456', '1'],
       ['CS001212000027', '448', '1'],
       ['CS001212000031', '296', '1'],
       ['CS001212000046', '228', '1'],
       ['CS001212000070', '456', '1']], dtype='<U21')

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [882]:
#cheat
df_customer['age_group'] = pd.cut(df_customer['age'], bins=[0, 10, 20, 30, 40, 50, 60, np.inf], right=False)
df_056 = df_customer[['customer_id', 'birth_day', 'age_group']].head(10)
df_056

,customer_id,birth_day,age_group
0,CS021313000114,1981-04-29,"[30.0, 40.0)"
1,CS037613000071,1952-04-01,"[60.0, inf)"
2,CS031415000172,1976-10-04,"[40.0, 50.0)"
3,CS028811000001,1933-03-27,"[60.0, inf)"
4,CS001215000145,1995-03-29,"[20.0, 30.0)"
5,CS020401000016,1974-09-15,"[40.0, 50.0)"
6,CS015414000103,1977-08-09,"[40.0, 50.0)"
7,CS029403000008,1973-08-17,"[40.0, 50.0)"
8,CS015804000004,1931-05-02,"[60.0, inf)"
9,CS033513000180,1962-07-11,"[50.0, 60.0)"


---
> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

df_056['age_gender'] = df_customer['gender'] + df_customer['age_group'].astype('str')
{ in enumerate(set(df_056['age_gender']))

In [895]:
df_056['age_gender'] = df_customer['gender'] + df_customer['age_group'].astype('str')
cat_dict = {cat:num for num, cat in enumerate(set(df_056['age_gender']))}
df_056.replace(cat_dict).head(10)

,customer_id,birth_day,age_group,age_gender
0,CS021313000114,1981-04-29,"[30.0, 40.0)",1
1,CS037613000071,1952-04-01,"[60.0, inf)",0
2,CS031415000172,1976-10-04,"[40.0, 50.0)",2
3,CS028811000001,1933-03-27,"[60.0, inf)",3
4,CS001215000145,1995-03-29,"[20.0, 30.0)",6
5,CS020401000016,1974-09-15,"[40.0, 50.0)",7
6,CS015414000103,1977-08-09,"[40.0, 50.0)",2
7,CS029403000008,1973-08-17,"[40.0, 50.0)",7
8,CS015804000004,1931-05-02,"[60.0, inf)",5
9,CS033513000180,1962-07-11,"[50.0, 60.0)",4


---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [912]:
gen_dum = pd.get_dummies(df_customer['gender_cd'])
pd.concat([df_customer['customer_id'], gen_dum], axis=1).head(10)

,customer_id,0,1,9
0,CS021313000114,0,1,0
1,CS037613000071,0,0,1
2,CS031415000172,0,1,0
3,CS028811000001,0,1,0
4,CS001215000145,0,1,0
5,CS020401000016,1,0,0
6,CS015414000103,0,1,0
7,CS029403000008,1,0,0
8,CS015804000004,1,0,0
9,CS033513000180,0,1,0


---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [926]:
from scipy.stats import zscore

df_receipt_059 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_059 = list(set(df_receipt_059['customer_id']))
customer_set_059.sort()

result_059 = []
for c_id in customer_set_059:
    c_am = df_receipt_059[df_receipt_059['customer_id'] == c_id]['amount']
    result_059.append([c_id, np.sum(c_am)])

result_059 = np.array(result_059)
c_sum = result_059[:,1].astype('int64')
c_z = zscore(c_sum)
result_059 = np.c_[result_059, c_z]
result_059[:10]

array([['CS001113000004', '1298', '-0.459377880258134'],
       ['CS001114000005', '626', '-0.7063903657165819'],
       ['CS001115000010', '3044', '0.18241348820979753'],
       ['CS001205000004', '1988', '-0.20574898893919197'],
       ['CS001205000006', '3337', '0.2901138724945077'],
       ['CS001211000025', '456', '-0.7688786432879153'],
       ['CS001212000027', '448', '-0.7718192681148016'],
       ['CS001212000031', '296', '-0.827691139825641'],
       ['CS001212000046', '228', '-0.8526864508541745'],
       ['CS001212000070', '456', '-0.7688786432879153']], dtype='<U32')

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [936]:
from sklearn import preprocessing

df_receipt_060 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_060 = list(set(df_receipt_060['customer_id']))
customer_set_060.sort()

result_060 = []
for c_id in customer_set_060:
    c_am = df_receipt_060[df_receipt_060['customer_id'] == c_id]['amount']
    result_060.append([c_id, np.sum(c_am)])

result_060 = np.array(result_060)
c_sum = result_060[:,1].astype('int64')

mn = preprocessing.MinMaxScaler()
c_mn = mn.fit_transform(c_sum[:,None])

result_060 = np.c_[result_060, c_mn]
result_060[:10]

array([['CS001113000004', '1298', '0.05335418839068474'],
       ['CS001114000005', '626', '0.02415710809871394'],
       ['CS001115000010', '3044', '0.12921445950643032'],
       ['CS001205000004', '1988', '0.08333333333333334'],
       ['CS001205000006', '3337', '0.14194473409801878'],
       ['CS001211000025', '456', '0.016770941953423706'],
       ['CS001212000027', '448', '0.016423357664233577'],
       ['CS001212000031', '296', '0.009819256169621133'],
       ['CS001212000046', '228', '0.00686478971150504'],
       ['CS001212000070', '456', '0.016770941953423706']], dtype='<U32')

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [938]:
df_receipt_061 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_061 = list(set(df_receipt_061['customer_id']))
customer_set_061.sort()

result_061 = []
for c_id in customer_set_061:
    c_am = df_receipt_061[df_receipt_061['customer_id'] == c_id]['amount']
    result_061.append([c_id, np.sum(c_am)])

result_061 = np.array(result_061)
c_sum = result_061[:,1].astype('int64')
c_log10 = np.log10(c_sum)

result_061 = np.c_[result_061, c_log10]
result_061[:10]

array([['CS001113000004', '1298', '3.1132746924643504'],
       ['CS001114000005', '626', '2.7965743332104296'],
       ['CS001115000010', '3044', '3.4834446480985353'],
       ['CS001205000004', '1988', '3.2984163800612945'],
       ['CS001205000006', '3337', '3.5233562066547925'],
       ['CS001211000025', '456', '2.6589648426644352'],
       ['CS001212000027', '448', '2.651278013998144'],
       ['CS001212000031', '296', '2.4712917110589387'],
       ['CS001212000046', '228', '2.357934847000454'],
       ['CS001212000070', '456', '2.6589648426644352']], dtype='<U32')

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [942]:
df_receipt_062 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_062 = list(set(df_receipt_062['customer_id']))
customer_set_062.sort()

result_062 = []
for c_id in customer_set_062:
    c_am = df_receipt_062[df_receipt_062['customer_id'] == c_id]['amount']
    result_062.append([c_id, np.sum(c_am)])

result_062 = np.array(result_062)
c_sum = result_062[:,1].astype('int64')
c_log = np.log(c_sum)

result_062 = np.c_[result_062, c_log]
result_062[:10]

array([['CS001113000004', '1298', '7.168579897264035'],
       ['CS001114000005', '626', '6.439350371100098'],
       ['CS001115000010', '3044', '8.020927718981577'],
       ['CS001205000004', '1988', '7.59488438721652'],
       ['CS001205000006', '3337', '8.112827478751374'],
       ['CS001211000025', '456', '6.1224928095143865'],
       ['CS001212000027', '448', '6.104793232414985'],
       ['CS001212000031', '296', '5.69035945432406'],
       ['CS001212000046', '228', '5.429345628954441'],
       ['CS001212000070', '456', '6.1224928095143865']], dtype='<U32')

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [944]:
df_product_tmp = df_product.copy()
df_product_tmp['befit'] = df_product_tmp['unit_price'] - df_product_tmp['unit_cost']
df_product_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,befit
0,P040101001,04,0401,040101,198.0,149.0,49.0
1,P040101002,04,0401,040101,218.0,164.0,54.0
2,P040101003,04,0401,040101,230.0,173.0,57.0
3,P040101004,04,0401,040101,248.0,186.0,62.0
4,P040101005,04,0401,040101,268.0,201.0,67.0
5,P040101006,04,0401,040101,298.0,224.0,74.0
6,P040101007,04,0401,040101,338.0,254.0,84.0
7,P040101008,04,0401,040101,420.0,315.0,105.0
8,P040101009,04,0401,040101,498.0,374.0,124.0
9,P040101010,04,0401,040101,580.0,435.0,145.0


---
> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [968]:
df_product_tmp = df_product.copy()
df_product_tmp['benefit_rate'] = (df_product_tmp['unit_price'] - df_product_tmp['unit_cost']) / df_product_tmp['unit_price']

rate = np.array(df_product_tmp['benefit_rate'])
rate[np.isnan(rate)] = 0
np.mean(rate)
#?

0.24894004069703796

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [988]:
df_product_tmp = df_product.copy()
df_product_tmp['new_unit_price'] = np.floor(df_product_tmp['unit_cost'] / 0.7)
df_product_tmp['benefit_rate'] = (df_product_tmp['new_unit_price'] - df_product_tmp['unit_cost']) / df_product_tmp['new_unit_price']
df_product_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_unit_price,benefit_rate
0,P040101001,04,0401,040101,198.0,149.0,212.0,0.297170
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,265.0,0.298113
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,362.0,0.298343
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


---
> P-066: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を四捨五入すること（0.5については偶数方向の丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [989]:
df_product_tmp = df_product.copy()
df_product_tmp['new_unit_price'] = np.round(df_product_tmp['unit_cost'] / 0.7)
df_product_tmp['benefit_rate'] = (df_product_tmp['new_unit_price'] - df_product_tmp['unit_cost']) / df_product_tmp['new_unit_price']
df_product_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_unit_price,benefit_rate
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,234.0,0.299145
2,P040101003,04,0401,040101,230.0,173.0,247.0,0.299595
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,287.0,0.299652
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,450.0,0.300000
8,P040101009,04,0401,040101,498.0,374.0,534.0,0.299625
9,P040101010,04,0401,040101,580.0,435.0,621.0,0.299517


---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [990]:
df_product_tmp = df_product.copy()
df_product_tmp['new_unit_price'] = np.ceil(df_product_tmp['unit_cost'] / 0.7)
df_product_tmp['benefit_rate'] = (df_product_tmp['new_unit_price'] - df_product_tmp['unit_cost']) / df_product_tmp['new_unit_price']
df_product_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,new_unit_price,benefit_rate
0,P040101001,04,0401,040101,198.0,149.0,213.0,0.300469
1,P040101002,04,0401,040101,218.0,164.0,235.0,0.302128
2,P040101003,04,0401,040101,230.0,173.0,248.0,0.302419
3,P040101004,04,0401,040101,248.0,186.0,266.0,0.300752
4,P040101005,04,0401,040101,268.0,201.0,288.0,0.302083
5,P040101006,04,0401,040101,298.0,224.0,320.0,0.300000
6,P040101007,04,0401,040101,338.0,254.0,363.0,0.300275
7,P040101008,04,0401,040101,420.0,315.0,451.0,0.301552
8,P040101009,04,0401,040101,498.0,374.0,535.0,0.300935
9,P040101010,04,0401,040101,580.0,435.0,622.0,0.300643


---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [992]:
df_product_tmp = df_product.copy()
df_product_tmp['add_tax'] = np.floor(df_product_tmp['unit_price'] * 1.1)
df_product_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,add_tax
0,P040101001,04,0401,040101,198.0,149.0,217.0
1,P040101002,04,0401,040101,218.0,164.0,239.0
2,P040101003,04,0401,040101,230.0,173.0,253.0
3,P040101004,04,0401,040101,248.0,186.0,272.0
4,P040101005,04,0401,040101,268.0,201.0,294.0
5,P040101006,04,0401,040101,298.0,224.0,327.0
6,P040101007,04,0401,040101,338.0,254.0,371.0
7,P040101008,04,0401,040101,420.0,315.0,462.0
8,P040101009,04,0401,040101,498.0,374.0,547.0
9,P040101010,04,0401,040101,580.0,435.0,638.0


---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [1005]:
df_069_0 = pd.merge(df_receipt, df_product, on='product_cd', how='inner')
df_069_1 = df_069_0[df_069_0['category_major_cd'] == '07'] 
customer_list_069 = list(set(df_069_1['customer_id']))
customer_list_069.sort()

result_069 = []
for c_id in customer_list_069:
    c_am_all = df_069_0[df_069_0['customer_id'] == c_id]['amount']
    c_am_07 = df_069_1[df_069_1['customer_id'] == c_id]['amount']
    result_069.append([c_id, np.sum(c_am_all), np.sum(c_am_07), np.sum(c_am_07)/np.sum(c_am_all)])
    
result_069 = np.array(result_069)
result_069[:10]    

array([['CS001113000004', '1298', '1298', '1.0'],
       ['CS001114000005', '626', '486', '0.7763578274760383'],
       ['CS001115000010', '3044', '2694', '0.8850197109067017'],
       ['CS001205000004', '1988', '346', '0.17404426559356137'],
       ['CS001205000006', '3337', '2004', '0.600539406652682'],
       ['CS001212000027', '448', '200', '0.44642857142857145'],
       ['CS001212000031', '296', '296', '1.0'],
       ['CS001212000046', '228', '108', '0.47368421052631576'],
       ['CS001212000070', '456', '308', '0.6754385964912281'],
       ['CS001213000018', '243', '145', '0.5967078189300411']],
      dtype='<U20')

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [1022]:
df_070 = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_070 = df_070.assign(sales_ymd = df_070['sales_ymd'].astype('str').astype('datetime64'))
df_070 = df_070.assign(application_date = df_070['application_date'].astype('datetime64'))
df_070['date_diff'] = df_070['sales_ymd'] - df_070['application_date']
df_070 = df_070[['customer_id', 'sales_ymd', 'application_date', 'date_diff']].drop_duplicates() #cheat
df_070.head(10)

,customer_id,sales_ymd,application_date,date_diff
0,CS006214000001,2018-11-03,2015-02-01,1371 days
1,CS006214000001,2017-05-09,2015-02-01,828 days
2,CS006214000001,2017-06-08,2015-02-01,858 days
4,CS006214000001,2018-10-28,2015-02-01,1365 days
7,CS006214000001,2019-09-08,2015-02-01,1680 days
8,CS006214000001,2018-01-31,2015-02-01,1095 days
9,CS006214000001,2017-07-05,2015-02-01,885 days
10,CS006214000001,2018-11-10,2015-02-01,1378 days
12,CS006214000001,2019-04-10,2015-02-01,1529 days
15,CS006214000001,2019-06-01,2015-02-01,1581 days


---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [1035]:
df_071 = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_071 = df_071.assign(sales_ymd = df_071['sales_ymd'].astype('str').astype('datetime64'))
df_071 = df_071.assign(application_date = df_071['application_date'].astype('datetime64'))
df_071['month_diff'] = df_071[['sales_ymd', 'application_date']].apply(lambda x: 
                                    relativedelta(x[0], x[1]).years * 12 + relativedelta(x[0], x[1]).months, axis=1)#cheat
df_071 = df_071[['customer_id', 'sales_ymd', 'application_date', 'month_diff']].drop_duplicates() #cheat
df_071.sort_values('customer_id').head(10)# cheat

,customer_id,sales_ymd,application_date,month_diff
60376,CS001113000004,2019-03-08,2015-11-05,40
20158,CS001114000005,2019-07-31,2016-04-12,39
20156,CS001114000005,2018-05-03,2016-04-12,24
29140,CS001115000010,2019-04-05,2015-04-17,47
29141,CS001115000010,2018-07-01,2015-04-17,38
29142,CS001115000010,2017-12-28,2015-04-17,32
6526,CS001205000004,2019-06-25,2016-06-15,36
6521,CS001205000004,2019-03-12,2016-06-15,32
6518,CS001205000004,2018-08-21,2016-06-15,26
6520,CS001205000004,2017-09-14,2016-06-15,14


---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [1037]:
df_072 = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_072 = df_072.assign(sales_ymd = df_072['sales_ymd'].astype('str').astype('datetime64'))
df_072 = df_072.assign(application_date = df_072['application_date'].astype('datetime64'))
df_072['year_diff'] = df_072[['sales_ymd', 'application_date']].apply(lambda x: relativedelta(x[0], x[1]).years, axis=1)
df_072 = df_072[['customer_id', 'sales_ymd', 'application_date', 'year_diff']]
df_072.head(10)

,customer_id,sales_ymd,application_date,year_diff
0,CS006214000001,2018-11-03,2015-02-01,3
1,CS006214000001,2017-05-09,2015-02-01,2
2,CS006214000001,2017-06-08,2015-02-01,2
3,CS006214000001,2017-06-08,2015-02-01,2
4,CS006214000001,2018-10-28,2015-02-01,3
5,CS006214000001,2018-10-28,2015-02-01,3
6,CS006214000001,2017-05-09,2015-02-01,2
7,CS006214000001,2019-09-08,2015-02-01,4
8,CS006214000001,2018-01-31,2015-02-01,2
9,CS006214000001,2017-07-05,2015-02-01,2


---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [1050]:
df_073 = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')
df_073 = df_073.assign(sales_ymd = df_073['sales_ymd'].astype('str').astype('datetime64').dt.tz_localize('Asia/Tokyo'))
df_073 = df_073.assign(application_date = df_073['application_date'].astype('datetime64').dt.tz_localize('Asia/Tokyo'))
df_073['epoch_diff'] = df_073['sales_ymd'].astype('int64') / 10**9 - df_073['application_date'].astype('int64') / 10**9
df_073[['customer_id', 'sales_ymd', 'application_date', 'epoch_diff']].drop_duplicates().head(10)

,customer_id,sales_ymd,application_date,epoch_diff
0,CS006214000001,2018-11-03 00:00:00+09:00,2015-02-01 00:00:00+09:00,118454400.0
1,CS006214000001,2017-05-09 00:00:00+09:00,2015-02-01 00:00:00+09:00,71539200.0
2,CS006214000001,2017-06-08 00:00:00+09:00,2015-02-01 00:00:00+09:00,74131200.0
4,CS006214000001,2018-10-28 00:00:00+09:00,2015-02-01 00:00:00+09:00,117936000.0
7,CS006214000001,2019-09-08 00:00:00+09:00,2015-02-01 00:00:00+09:00,145152000.0
8,CS006214000001,2018-01-31 00:00:00+09:00,2015-02-01 00:00:00+09:00,94608000.0
9,CS006214000001,2017-07-05 00:00:00+09:00,2015-02-01 00:00:00+09:00,76464000.0
10,CS006214000001,2018-11-10 00:00:00+09:00,2015-02-01 00:00:00+09:00,119059200.0
12,CS006214000001,2019-04-10 00:00:00+09:00,2015-02-01 00:00:00+09:00,132105600.0
15,CS006214000001,2019-06-01 00:00:00+09:00,2015-02-01 00:00:00+09:00,136598400.0


---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [1075]:
import datetime

df_074 = df_receipt.copy()
df_074 = df_074.assign(sales_ymd = df_074['sales_ymd'].astype('str').astype('datetime64').dt.tz_localize('Asia/Tokyo'))
df_074['week'] = df_074['sales_ymd'].dt.weekday
df_074['monday_date'] = df_074[['sales_ymd', 'week']].apply(lambda x: x[0] - datetime.timedelta(days=x[1]), axis=1)
df_074[['sales_ymd', 'week', 'monday_date']].head(10)

,sales_ymd,week,monday_date
0,2018-11-03 00:00:00+09:00,5,2018-10-29 00:00:00+09:00
1,2018-11-18 00:00:00+09:00,6,2018-11-12 00:00:00+09:00
2,2017-07-12 00:00:00+09:00,2,2017-07-10 00:00:00+09:00
3,2019-02-05 00:00:00+09:00,1,2019-02-04 00:00:00+09:00
4,2018-08-21 00:00:00+09:00,1,2018-08-20 00:00:00+09:00
5,2019-06-05 00:00:00+09:00,2,2019-06-03 00:00:00+09:00
6,2018-12-05 00:00:00+09:00,2,2018-12-03 00:00:00+09:00
7,2019-09-22 00:00:00+09:00,6,2019-09-16 00:00:00+09:00
8,2017-05-04 00:00:00+09:00,3,2017-05-01 00:00:00+09:00
9,2019-10-10 00:00:00+09:00,3,2019-10-07 00:00:00+09:00


---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [1077]:
df_customer.sample(int(df_customer.shape[0]*0.01)).head(10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
9357,CS001615000049,吉永 奈央,1,女性,1950-03-28,69,144-0056,東京都大田区西六郷**********,S13001,20150306,0-00000000-0
7947,CS003215000043,二宮 遥,1,女性,1993-09-03,25,201-0002,東京都狛江市東野川**********,S13003,20150912,A-20090818-B
4140,CS041713000001,菊地 茜,1,女性,1939-06-11,79,206-0015,東京都多摩市落川**********,S13041,20161026,0-00000000-0
18739,CS006512000216,織田 幸子,1,女性,1966-05-26,52,224-0045,神奈川県横浜市都筑区東方町**********,S14006,20180509,0-00000000-0
9211,CS005414000033,堀 真悠子,1,女性,1977-03-02,42,167-0022,東京都杉並区下井草**********,S13005,20150320,C-20101006-C
15124,CS029415000051,多岐川 季衣,1,女性,1972-03-13,47,279-0003,千葉県浦安市海楽**********,S12029,20150413,D-20101021-D
10307,CS005315000060,佐久間 郁恵,1,女性,1983-08-27,35,167-0031,東京都杉並区本天沼**********,S13005,20151011,4-20090911-8
18192,CS027214000015,中村 さとみ,1,女性,1994-05-23,24,251-0021,神奈川県藤沢市鵠沼神明**********,S14027,20150715,D-20100402-D
6746,CS022401000019,三浦 光博,0,男性,1969-06-10,49,240-0113,神奈川県三浦郡葉山町長柄**********,S14022,20150822,0-00000000-0
8528,CS034503000051,高柳 陽子,9,不明,1960-08-12,58,216-0001,神奈川県川崎市宮前区野川**********,S14034,20151109,0-00000000-0


---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [1090]:
from sklearn.model_selection import train_test_split

data10 = train_test_split(df_customer['gender_cd'], test_size = 0.1)[1]
data10 = data10.value_counts()
data10

1    1787
0     301
9     110
Name: gender_cd, dtype: int64

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [1120]:
df_receipt_077 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_077 = list(set(df_receipt_077['customer_id']))
customer_set_077.sort()

result_077 = []
for c_id in customer_set_077:
    c_am = df_receipt_077[df_receipt_077['customer_id'] == c_id]['amount']
    result_077.append([c_id, np.sum(c_am)])

result_077 = np.array(result_077)
c_sum = result_077[:,1].astype('int64')
result_077[np.abs(c_sum - np.mean(c_sum)) > np.std(c_sum) * 3][:10]

array([['CS001605000009', '18925'],
       ['CS006415000147', '12723'],
       ['CS006515000023', '18372'],
       ['CS006515000125', '12575'],
       ['CS006515000209', '11373'],
       ['CS007115000006', '11528'],
       ['CS007514000056', '13293'],
       ['CS007514000094', '15735'],
       ['CS007515000107', '11188'],
       ['CS007615000026', '11959']], dtype='<U14')

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [1123]:
df_receipt_078 = df_receipt[df_receipt['customer_id'].str.contains('[A-Y]...........')]
customer_set_078 = list(set(df_receipt_078['customer_id']))
customer_set_078.sort()

result_078 = []
for c_id in customer_set_078:
    c_am = df_receipt_078[df_receipt_078['customer_id'] == c_id]['amount']
    result_078.append([c_id, np.sum(c_am)])

result_078 = np.array(result_078)
c_sum = result_078[:,1].astype('int64')

c_per = np.percentile(c_sum, q=[25, 75])
IQR = c_per[1] - c_per[0]
result_078[np.logical_or(c_per[0]-(1.5*IQR) >= c_sum, c_per[1]+(1.5*IQR) <= c_sum)][:10]

array([['CS001414000048', '8584'],
       ['CS001605000009', '18925'],
       ['CS002415000594', '9568'],
       ['CS004414000181', '9584'],
       ['CS005415000137', '8734'],
       ['CS006414000001', '9156'],
       ['CS006414000029', '9179'],
       ['CS006415000105', '10042'],
       ['CS006415000147', '12723'],
       ['CS006415000157', '10648']], dtype='<U14')

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [1126]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [1130]:
df_product_1 = df_product.dropna(how="any")
{'df_product':df_product.shape, 'df_product_1':df_product_1.shape}

{'df_product': (10030, 6), 'df_product_1': (10023, 6)}

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [1135]:
df_product_2 = df_product.fillna(np.round(df_product.mean()))
df_product_2.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [1136]:
df_product_3 = df_product.fillna(np.round(df_product.median()))
df_product_3.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [1143]:
small_cat_set = list(set(df_product['category_small_cd']))
small_cat_set.sort()

for s_c in small_cat_set:
    

SyntaxError: unexpected EOF while parsing (<ipython-input-1143-71daafacee3c>, line 5)

---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [1165]:
df_084_0 = df_customer.fillna(0)
df_084_1 = df_receipt.fillna(0)
customer_set_084 = list(set(df_084_0['customer_id']))
customer_set_084.sort()

df_084_1_2019 = df_084_1[(df_084_1['sales_ymd'] >= 20190101) & (df_084_1['sales_ymd'] <= 20191231)]

result_084 = []
for c_id in customer_set_084:
    c_am_2019 = np.sum(df_084_1_2019[df_084_1_2019['customer_id'] == c_id]['amount'])
    c_am_all = np.sum(df_084_1[df_084_1['customer_id'] == c_id]['amount'])
    result_084.append([c_id, c_am_2019, c_am_all, c_am_2019/c_am_all])
    
result_084 = np.array(result_084)

c_rate = result_084[:,3].astype('float32')
c_rate[np.isnan(c_rate)] = 0
result_084[:,3] = c_rate
result_084[result_084[:,3] != '0.0'][:10]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':


array([['CS001113000004', '1298', '1298', '1.0'],
       ['CS001114000005', '188', '626', '0.3003195'],
       ['CS001115000010', '578', '3044', '0.18988174'],
       ['CS001205000004', '702', '1988', '0.35311872'],
       ['CS001205000006', '486', '3337', '0.14563979'],
       ['CS001211000025', '456', '456', '1.0'],
       ['CS001212000070', '456', '456', '1.0'],
       ['CS001214000009', '664', '4685', '0.14172892'],
       ['CS001214000017', '2962', '4132', '0.71684414'],
       ['CS001214000048', '1889', '2374', '0.7957035']], dtype='<U20')

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [1175]:
#cheat
df_customer_1 = pd.merge(df_customer[['customer_id', 'postal_cd']],
                         df_geocode[['postal_cd', 'longitude' ,'latitude']],
                         how='inner', on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id'). \
    agg({'longitude':'mean', 'latitude':'mean'}).reset_index(). \
    rename(columns={'longitude':'m_longitude', 'latitude':'m_latitude'})

df_customer_1 = pd.merge(df_customer, df_customer_1, how='inner', on='customer_id')
df_customer_1.head(3)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,m_longitude,m_latitude
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.31779,35.41358
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.83502,35.67193
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.68965,35.67374


---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [1192]:
df_store_tmp = df_store.copy()
df_store_tmp = df_store_tmp.rename(columns={'store_cd':'application_store_cd'})
df_086 = pd.merge(df_customer_1, df_store_tmp, on='application_store_cd', how='inner')
df_086[['m_longitude', 'm_latitude', 'longitude', 'latitude']]
df_086['distance'] = 6371*np.arccos(np.sin(np.deg2rad(df_086['latitude']))*np.sin(np.deg2rad(df_086['m_latitude'])) + \
                                   np.cos(np.deg2rad(df_086['latitude']))*np.cos(np.deg2rad(df_086['m_latitude'])) *\
                                   np.cos(np.deg2rad(df_086['longitude']) - np.deg2rad(df_086['m_longitude'])))
df_086[['customer_id', 'address_x', 'address_y', 'distance']].head(10)

,customer_id,address_x,address_y,distance
0,CS021313000114,神奈川県伊勢原市粟窪**********,神奈川県伊勢原市伊勢原四丁目,1.394409
1,CS021313000025,神奈川県伊勢原市伊勢原**********,神奈川県伊勢原市伊勢原四丁目,0.474282
2,CS021411000096,神奈川県伊勢原市高森**********,神奈川県伊勢原市伊勢原四丁目,2.480155
3,CS021415000150,神奈川県伊勢原市上粕屋**********,神奈川県伊勢原市伊勢原四丁目,2.734723
4,CS021313000046,神奈川県伊勢原市池端**********,神奈川県伊勢原市伊勢原四丁目,1.111911
5,CS021103000002,神奈川県伊勢原市西富岡**********,神奈川県伊勢原市伊勢原四丁目,2.384941
6,CS021214000028,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.399344
7,CS021512000095,神奈川県伊勢原市沼目**********,神奈川県伊勢原市伊勢原四丁目,1.993991
8,CS021613000002,神奈川県伊勢原市桜台**********,神奈川県伊勢原市伊勢原四丁目,1.399344
9,CS021412000147,神奈川県伊勢原市三ノ宮**********,神奈川県伊勢原市伊勢原四丁目,3.507680


---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [1206]:
#cheat
df_tmp = df_receipt.groupby('customer_id').agg({'amount':sum}).reset_index()
df_customer_u = pd.merge(df_customer, df_tmp, how='left', on='customer_id').sort_values(['amount', 'customer_id']
                                                                                        , ascending=[False, True])
df_customer_u.drop_duplicates(subset=['customer_name', 'postal_cd'], keep='first', inplace=True)

print('減少数: ', len(df_customer) - len(df_customer_u))

減少数:  30


---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [ ]:
#cheat
df_customer_n = pd.merge(df_customer, df_customer_u[['customer_name', 'postal_cd', 'customer_id']],
                        how='inner', on =['customer_name', 'postal_cd'])
df_customer_n.rename(columns={'customer_id_x':'customer_id', 'customer_id_y':'integration_id'}, inplace=True)

print('ID数の差', len(df_customer_n['customer_id'].unique()) - len(df_customer_n['integration_id'].unique()))

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [ ]:
del df_customer_1
del df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [1227]:
from sklearn.model_selection import train_test_split

df_089 = pd.merge(df_customer, df_receipt['customer_id'], how='inner', on='customer_id')#cheat

train, test = train_test_split(df_089, test_size=0.2)
print("train:"+str(train.shape[0]/df_089.shape[0])+"\ntest:"+str(test.shape[0]/df_089.shape[0]))

train:0.7999908650771901
test:0.2000091349228099


---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [1255]:
date_list = np.array([['201'+str(y)+'{:02}'.format(m) for m in range(1,13)] for y in range(7,10)]).reshape((36))[:-2]
df_090 = df_receipt.copy()

date_amount = []
for date in date_list:
    c_am = np.sum(df_090[df_090['sales_ymd'].astype('str').str.startswith(date)]['amount'])
    date_amount.append([date, c_am])
    
date_amount = np.array(date_amount)
tra_0, tes_0 = date_amount[:12], date_amount[12:18]
tra_1, tes_1 = date_amount[1:13], date_amount[13:19]
tra_2, tes_2 = date_amount[2:14], date_amount[14:20]
tra_0

array([['201701', '902056'],
       ['201702', '764413'],
       ['201703', '962945'],
       ['201704', '847566'],
       ['201705', '884010'],
       ['201706', '894242'],
       ['201707', '959205'],
       ['201708', '954836'],
       ['201709', '902037'],
       ['201710', '905739'],
       ['201711', '932157'],
       ['201712', '939654']], dtype='<U7')

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [1305]:
df_091 = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_091 = pd.merge(df_customer, df_091, how='left', on='customer_id')

df_091 = df_091.fillna(0)
df_091_0 = df_091[df_091['amount'] != 0]
df_091_1 = df_091[df_091['amount'] == 0]

data_l = [df_091_0, df_091_1]

data_sizes = np.array([df_091_0.shape[0], df_091_1.shape[0]])
under = np.min(data_sizes)
upper = np.max(data_sizes)
under_class = np.argmin(data_sizes)
upper_class = np.argmax(data_sizes)

data_0 = data_l[under_class]
data_1 = data_l[upper_class].iloc[np.random.randint(0, upper, under),:]
data_1.shape[0]

8306

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [1306]:
#cheat
df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_customer_s = df_customer.drop(columns='gender')

,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
...,...,...,...,...,...,...,...,...,...,...
21966,CS002512000474,市村 夏希,1,1959-10-12,59,185-0034,東京都国分寺市光町**********,S13002,20171110,0-00000000-0
21967,CS029414000065,上村 怜奈,1,1970-10-19,48,279-0043,千葉県浦安市富士見**********,S12029,20150313,F-20101028-F
21968,CS012403000043,堀越 明,0,1972-12-16,46,231-0825,神奈川県横浜市中区本牧間門**********,S14012,20150406,0-00000000-0
21969,CS033512000184,池谷 華子,1,1964-06-05,54,245-0016,神奈川県横浜市泉区和泉町**********,S14033,20160206,0-00000000-0


---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [1312]:
df_093 = pd.merge(df_product, df_category, on='category_medium_cd', how='inner')
df_093

,product_cd,category_major_cd_x,category_medium_cd,category_small_cd_x,unit_price,unit_cost,category_major_cd_y,category_major_name,category_medium_name,category_small_cd_y,category_small_name
0,P040101001,04,0401,040101,198.0,149.0,04,惣菜,御飯類,040101,弁当類
1,P040101001,04,0401,040101,198.0,149.0,04,惣菜,御飯類,040102,寿司類
2,P040101002,04,0401,040101,218.0,164.0,04,惣菜,御飯類,040101,弁当類
3,P040101002,04,0401,040101,218.0,164.0,04,惣菜,御飯類,040102,寿司類
4,P040101003,04,0401,040101,230.0,173.0,04,惣菜,御飯類,040101,弁当類
...,...,...,...,...,...,...,...,...,...,...,...
62168,P091503003,09,0915,091503,1080.0,810.0,09,洗剤,ＤＩＹ用品,091503,園芸用品
62169,P091503004,09,0915,091503,1130.0,848.0,09,洗剤,ＤＩＹ用品,091501,建築・塗装材料
62170,P091503004,09,0915,091503,1130.0,848.0,09,洗剤,ＤＩＹ用品,091503,園芸用品
62171,P091503005,09,0915,091503,1280.0,960.0,09,洗剤,ＤＩＹ用品,091501,建築・塗装材料


---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [1317]:
df_093.to_csv('data/df_094.csv', encoding='UTF-8')

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [1315]:
df_093.to_csv('data/df_095.csv', encoding='CP932')

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [1318]:
df_093.to_csv('data/df_096.csv', encoding='UTF-8', header=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [1321]:
df_097 = pd.read_csv('data/df_094.csv')
df_097.head(10)

,Unnamed: 0,product_cd,category_major_cd_x,category_medium_cd,category_small_cd_x,unit_price,unit_cost,category_major_cd_y,category_major_name,category_medium_name,category_small_cd_y,category_small_name
0,0,P040101001,4,401,40101,198.0,149.0,4,惣菜,御飯類,40101,弁当類
1,1,P040101001,4,401,40101,198.0,149.0,4,惣菜,御飯類,40102,寿司類
2,2,P040101002,4,401,40101,218.0,164.0,4,惣菜,御飯類,40101,弁当類
3,3,P040101002,4,401,40101,218.0,164.0,4,惣菜,御飯類,40102,寿司類
4,4,P040101003,4,401,40101,230.0,173.0,4,惣菜,御飯類,40101,弁当類
5,5,P040101003,4,401,40101,230.0,173.0,4,惣菜,御飯類,40102,寿司類
6,6,P040101004,4,401,40101,248.0,186.0,4,惣菜,御飯類,40101,弁当類
7,7,P040101004,4,401,40101,248.0,186.0,4,惣菜,御飯類,40102,寿司類
8,8,P040101005,4,401,40101,268.0,201.0,4,惣菜,御飯類,40101,弁当類
9,9,P040101005,4,401,40101,268.0,201.0,4,惣菜,御飯類,40102,寿司類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [1323]:
df_098 = pd.read_csv('data/df_096.csv')
df_098.head(10)

,0,P040101001,04,0401,040101,198.0,149.0,04.1,惣菜,御飯類,040101.1,弁当類
0,1,P040101001,4,401,40101,198.0,149.0,4,惣菜,御飯類,40102,寿司類
1,2,P040101002,4,401,40101,218.0,164.0,4,惣菜,御飯類,40101,弁当類
2,3,P040101002,4,401,40101,218.0,164.0,4,惣菜,御飯類,40102,寿司類
3,4,P040101003,4,401,40101,230.0,173.0,4,惣菜,御飯類,40101,弁当類
4,5,P040101003,4,401,40101,230.0,173.0,4,惣菜,御飯類,40102,寿司類
5,6,P040101004,4,401,40101,248.0,186.0,4,惣菜,御飯類,40101,弁当類
6,7,P040101004,4,401,40101,248.0,186.0,4,惣菜,御飯類,40102,寿司類
7,8,P040101005,4,401,40101,268.0,201.0,4,惣菜,御飯類,40101,弁当類
8,9,P040101005,4,401,40101,268.0,201.0,4,惣菜,御飯類,40102,寿司類
9,10,P040101006,4,401,40101,298.0,224.0,4,惣菜,御飯類,40101,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [1324]:
df_093.to_csv('data/df_099.tsv', encoding='UTF-8', sep='\t')

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [1327]:
df_100 = pd.read_csv('data/df_099.tsv', sep='\t')
df_100.head(10)

,Unnamed: 0,product_cd,category_major_cd_x,category_medium_cd,category_small_cd_x,unit_price,unit_cost,category_major_cd_y,category_major_name,category_medium_name,category_small_cd_y,category_small_name
0,0,P040101001,4,401,40101,198.0,149.0,4,惣菜,御飯類,40101,弁当類
1,1,P040101001,4,401,40101,198.0,149.0,4,惣菜,御飯類,40102,寿司類
2,2,P040101002,4,401,40101,218.0,164.0,4,惣菜,御飯類,40101,弁当類
3,3,P040101002,4,401,40101,218.0,164.0,4,惣菜,御飯類,40102,寿司類
4,4,P040101003,4,401,40101,230.0,173.0,4,惣菜,御飯類,40101,弁当類
5,5,P040101003,4,401,40101,230.0,173.0,4,惣菜,御飯類,40102,寿司類
6,6,P040101004,4,401,40101,248.0,186.0,4,惣菜,御飯類,40101,弁当類
7,7,P040101004,4,401,40101,248.0,186.0,4,惣菜,御飯類,40102,寿司類
8,8,P040101005,4,401,40101,268.0,201.0,4,惣菜,御飯類,40101,弁当類
9,9,P040101005,4,401,40101,268.0,201.0,4,惣菜,御飯類,40102,寿司類


# これで１００本終わりです。おつかれさまでした！